In [4]:
import json
from tqdm import tqdm
import re
import torch
import pandas as pd
# 定义 color_mapping
color_mapping = {
    'red': [0.1, 0.15],
    'green': [0.15, 0.3],
    'blue': [0.3, 0.45],
    'yellow': [0.45, 0.6],
    'orange': [0.6, 0.75],
    'purple': [0.75, 0.9]
}

# 提取所有颜色
colors = color_mapping.keys()

# 检查颜色是否存在于字符串中
def find_colors_in_string(input_string):
    found_colors = [color for color in colors if color in input_string]
    return found_colors


def map_to_color(pixel):
    if pixel<0.1:
        return 'black'
    elif 0.1<=pixel<0.15:
        return 'red'
    elif 0.15<=pixel<0.3:
        return 'green'
    elif 0.3<=pixel<0.45:
        return 'blue'
    elif 0.45<=pixel<0.6:
        return 'yellow'
    elif 0.6<=pixel<0.75:
        return 'orange'
    elif 0.75<=pixel<=0.9:
        return 'purple'
    else:
        return 'other'

def compare_img(gen_img,gt_img):
    correct_pixel=0
    incorrect_pixel=0
    for i in range(len(gen_img)):
        for j in range(len(gen_img[i])):
            if map_to_color(gen_img[i][j])!=map_to_color(gt_img[i][j]):
                incorrect_pixel+=1
            else:
                correct_pixel+=1
    return correct_pixel,incorrect_pixel

acc=[]
answer_list=[i*30 for i in range(10,11)]
for answer in answer_list:
    #data_path=f'/datadrive_a/jihai/LLaVA/answer/answer-llava-v1.5-7b-mix-u-odd-{i}.jsonl'
    data_path=f'./answer/answer-llava-v1.5-7b-siglip-vq_u_weather_biased-2-u-sw-lora-{answer}.jsonl'
    time_count=0
    time_score=0
    weather_count=0
    weather_score=0
    position_count=0
    position_score=0
    battery_count=0
    battery_score=0
    with open (data_path, "r") as f:
        for line in tqdm(f):
            json_obj = json.loads(line.strip())
            ground_truth=json_obj['groun_truth']
            answer=json_obj['answer']
            prompt=json_obj['prompt']
            if prompt[0]!='<':
                continue
            if ' ' not in ground_truth:
                if ':' in ground_truth:
                    time_count+=1
                    pattern = r"(\d{2}):(\d{2}):(\d{2})"
                    match = re.search(pattern, ground_truth)
                    gt_h = int(match.group(1))
                    gt_m = int(match.group(2))
                    gt_s = int(match.group(3))
                    match = re.search(pattern, answer)
                    if match:
                        ans_h = int(match.group(1))
                        ans_m = int(match.group(2))
                        ans_s = int(match.group(3))
                        err_h=abs(ans_h - gt_h)
                        err_h=min(err_h,12-err_h)
                        err_m=abs(ans_m - gt_m)
                        err_m=min(err_m,60-err_m)
                        err_s=abs(ans_s - gt_s)
                        err_s=min(err_s,60-err_s)
                        err=(err_h/6.0 + err_m/30.0 + err_s/30.0)/3.0
    
                        time_score+=1-err
                elif 'sunny' in ground_truth or 'raining' in ground_truth or 'cloudy' in ground_truth:
                    weather_count+=1
                    if ground_truth in answer:
                        weather_score+=1
                elif '-' in ground_truth:
                    position_count+=1
                    if ground_truth in answer:
                        position_score+=1
                elif '%' in ground_truth:
                    battery_count+=1
                    match = re.search(r'\b(100|[1-9]\d?|0)%', ground_truth)
                    gt=int(match.group(1)) / 100
                    match = re.search(r'\b(100|[1-9]\d?|0)%', answer)
                    if match:
                        ans=int(match.group(1)) / 100
                        err=abs(ans - gt)
                        battery_score+=1-err
                else:
                    raise ValueError(f"Unknown ground truth format: {ground_truth}")
    time_acc=time_score/time_count
    weather_acc=weather_score/weather_count
    position_acc=position_score/position_count
    battery_acc=battery_score/battery_count
    print(time_count, time_score)
    print(weather_count, weather_score)
    print(position_count, position_score)
    print(battery_count, battery_score)
    total_acc=(time_score+weather_score+position_score+battery_score)/(time_count+weather_count+position_count+battery_count)
    acc.append([time_acc,weather_acc,position_acc,battery_acc,total_acc])
    print(f"time_acc: {time_acc:.4f}, weather_acc: {weather_acc:.4f}, position_acc: {position_acc:.4f}, battery_acc: {battery_acc:.4f}, total_acc: {total_acc:.4f}")
acc=torch.Tensor(acc)
acc=acc.permute(1,0)
print(f"time_acc:{acc[0]}")
print(f"weather_acc:{acc[1]}")
print(f"position_acc:{acc[2]}")
print(f"battery_acc:{acc[3]}")
print(f"total_acc:{acc[4]}")

columns = [f"{i*10}%" for i in range(1, 1+acc.shape[1])]
row_index = ['time_acc', 'weather_acc', 'position_acc', 'battery_acc', 'total_acc']
df = pd.DataFrame(data=acc.numpy(),
                  index=row_index,
                  columns=columns)
df.to_csv("output.csv", index=True, header=True)



400it [00:00, 121205.14it/s]

37 19.58888888888888
35 10
80 80
48 47.83
time_acc: 0.5294, weather_acc: 0.2857, position_acc: 1.0000, battery_acc: 0.9965, total_acc: 0.7871
time_acc:tensor([0.5294])
weather_acc:tensor([0.2857])
position_acc:tensor([1.])
battery_acc:tensor([0.9965])
total_acc:tensor([0.7871])


In [25]:
time_count=35
weather_count=31
position_count=61
battery_count=40
time_score=0.963*35
weather_score=1*31
position_score=1*61
battery_score=1*40

time_acc=time_score/time_count
weather_acc=weather_score/weather_count
position_acc=position_score/position_count
battery_acc=battery_score/battery_count
print(time_acc)
print(weather_acc)
print(position_acc)
print(battery_acc)
total_acc=(time_score+weather_score+position_score+battery_score)/(time_count+weather_count+position_count+battery_count)
print(total_acc)

0.963
1.0
1.0
1.0
0.9922455089820358


In [ ]:
import json
data_path='/public_data/jihai/data/multimodalout/smart_watch_train_120ku_180km.json'
data_list=json.load(open(data_path, "r"))
print(len(data_list))

In [1]:
import matplotlib.pyplot as plt
import random
import os
from PIL import Image, ImageDraw, ImageFont,ImageColor

# sample=torch.load('/public_data/jihai/understanding/scripts/v1_5/answer/answer-llava-v1.5-7b-siglip-vq-sw-lora-450-image/11.pt')
# print(sample.shape)
# plt.imshow(sample)

gt_image_folder='/public_data/jihai/data/multimodalout/smart_watch_image_test'

answer_list=[i*45 for i in range(1,18)]
answer=answer_list[-1]
#data_path=f'/datadrive_a/jihai/LLaVA/answer/answer-llava-v1.5-7b-mix-u-odd-{i}.jsonl'
data_path=f'./answer/answer-llava-v1.5-7b-vq-vq_120ku_180km-2-sw-lora-{answer}.jsonl'
count=0
time_count=0
weather_count=0
position_count=0
battery_count=0
with open (data_path, "r") as f:
    for line in tqdm(f):
        json_obj = json.loads(line.strip())
        ground_truth=json_obj['groun_truth']
        answer=json_obj['answer']
        prompt=json_obj['prompt']
        if prompt[0]=='<':
            continue
        if ':' in prompt:
            time_count+=1
            # pattern = r"(\d{2}):(\d{2}):(\d{2})"
            # match = re.search(pattern, prompt)
            # gt_h = int(match.group(1))
            # gt_m = int(match.group(2))
            # gt_s = int(match.group(3))
           
        if 'sunny' in prompt or 'raining' in prompt or 'cloudy' in prompt:
            weather_count+=1
            
        if '-' in prompt:
            position_count+=prompt.count('-')
           
        if '%' in prompt:
            battery_count+=1
            match = re.search(r'\b(100|[1-9]\d?|0)%', prompt)
            gt=int(match.group(1)) / 100
            
        gt_image_path=os.path.join(gt_image_folder,json_obj['groun_truth_img_tensor'])
        gt_image=Image.open(gt_image_path)
        gen_image=torch.load(json_obj['output_img_file'])
        
        # plt.title(prompt)
        # plt.imshow(gt_image)
        # plt.imshow(gen_image)
        fig, axes = plt.subplots(1, 2, figsize=(10, 5))  # figsize 可根据需要调整

        # 设置整个图的标题
        # fig.suptitle(prompt, fontsize=16)  # 总标题
        print(prompt)


        # 左边子图：显示 gt_image
        axes[0].imshow(gt_image)
        axes[0].set_title("Ground Truth Image")  # 子图标题
        axes[0].axis('off')  # 关闭坐标轴

        # 右边子图：显示 gen_image
        axes[1].imshow(gen_image)
        axes[1].set_title("Generated Image")  # 子图标题
        axes[1].axis('off')  # 关闭坐标轴

        # 调整布局以避免重叠
        plt.tight_layout(rect=[0, 0, 1, 0.9])  # 为总标题留出空间

        # 显示图像
        plt.savefig(f'./calculate/{count}.png')
        plt.close(fig)
        count+=1
        if count==30:
            break
print(f"time_count:{time_count}")
print(f"weather_count:{weather_count}")
print(f"position_count:{position_count}")
print(f"battery_count:{battery_count}")

NameError: name 'tqdm' is not defined

In [ ]:


class LazySupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_path: str,
                 tokenizer: transformers.PreTrainedTokenizer,
                 data_args: DataArguments):
        super(LazySupervisedDataset, self).__init__()
        list_data_dict = json.load(open(data_path, "r"))

        rank0_print("Formatting inputs...Skip in lazy mode")
        self.tokenizer = tokenizer
        self.list_data_dict = list_data_dict
        self.data_args = data_args

    def __len__(self):
        return len(self.list_data_dict)

    @property
    def lengths(self):
        length_list = []
        for sample in self.list_data_dict:
            img_tokens = 128 if 'image' in sample else 0
            length_list.append(sum(len(conv['value'].split()) for conv in sample['conversations']) + img_tokens)
        return length_list

    @property
    def modality_lengths(self):
        length_list = []
        for sample in self.list_data_dict:
            cur_len = sum(len(conv['value'].split()) for conv in sample['conversations'])
            cur_len = cur_len if 'image' in sample else -cur_len
            length_list.append(cur_len)
        return length_list

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        sources = self.list_data_dict[i]
        if isinstance(i, int):
            sources = [sources]
        assert len(sources) == 1, "Don't know why it is wrapped to a list"  # FIXME
        if 'image' in sources[0]:
            image_file = self.list_data_dict[i]['image']
            image_folder = self.data_args.image_folder
            processor = self.data_args.image_processor
            image = Image.open(os.path.join(image_folder, image_file)).convert('RGB')
            if self.data_args.image_aspect_ratio == 'pad':
                def expand2square(pil_img, background_color):
                    width, height = pil_img.size
                    if width == height:
                        return pil_img
                    elif width > height:
                        result = Image.new(pil_img.mode, (width, width), background_color)
                        result.paste(pil_img, (0, (width - height) // 2))
                        return result
                    else:
                        result = Image.new(pil_img.mode, (height, height), background_color)
                        result.paste(pil_img, ((height - width) // 2, 0))
                        return result
                image = expand2square(image, tuple(int(x*255) for x in processor.image_mean))
                image = processor.preprocess(image, return_tensors='pt')['pixel_values'][0]
            else:
                image = processor.preprocess(image, return_tensors='pt')['pixel_values'][0]
            sources = preprocess_multimodal(
                copy.deepcopy([e["conversations"] for e in sources]),
                self.data_args)
        else:
            sources = copy.deepcopy([e["conversations"] for e in sources])
        data_dict = preprocess(
            sources,
            self.tokenizer,
            has_image=('image' in self.list_data_dict[i]))
        if isinstance(i, int):
            data_dict = dict(input_ids=data_dict["input_ids"][0],
                             labels=data_dict["labels"][0])

        # image exist in the data
        if 'image' in self.list_data_dict[i]:
            data_dict['image'] = image
        elif self.data_args.is_multimodal:
            # image does not exist in the data, but the model is multimodal
            crop_size = self.data_args.image_processor.crop_size
            data_dict['image'] = torch.zeros(3, crop_size['height'], crop_size['width'])
        return data_dict


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances]
                                  for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids,
            batch_first=True,
            padding_value=self.tokenizer.pad_token_id)
        labels = torch.nn.utils.rnn.pad_sequence(labels,
                                                 batch_first=True,
                                                 padding_value=IGNORE_INDEX)
        input_ids = input_ids[:, :self.tokenizer.model_max_length]
        labels = labels[:, :self.tokenizer.model_max_length]
        batch = dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

        if 'image' in instances[0]:
            images = [instance['image'] for instance in instances]
            if all(x is not None and x.shape == images[0].shape for x in images):
                batch['images'] = torch.stack(images)
            else:
                batch['images'] = images

        return batch

In [13]:
list_data_dict = json.load(open('/public_data/ShareGPT4V/sharegpt4v/share-captioner_coco_lcs_sam_1246k_1107.json', "r"))
print(list_data_dict[0])

{'id': 'sa_545504', 'image': 'sam/images/sa_545504.jpg', 'conversations': [{'from': 'human', 'value': "<image>\nWhat is this photo about'?"}, {'from': 'gpt', 'value': "The image captures a breathtaking view of the Hong Kong skyline at sunset. The sky, awash with hues of orange and blue, serves as a stunning backdrop to the city's architectural marvels. The tallest building, the International Finance Centre, pierces the sky with its imposing height. Its lights are switched on, casting a warm glow that contrasts with the cool tones of the evening sky.\n\nThe other buildings, though not as tall, are no less impressive. They are adorned with lights that twinkle like stars against the twilight sky. The water below mirrors the sky's colors, adding to the overall vibrancy of the scene.\n\nThe perspective of the image is from the water, looking towards the shore. This viewpoint allows for a comprehensive view of the cityscape, from the towering skyscrapers to the smaller structures nestled amo

In [14]:
import random
from tqdm import tqdm
generation_q=[
        "Can you generate a image of the following:",
        "Can you draw a image of the following:",
        "Please show a image of the following:"]

generation_a=[
        "Sure, here is the image. \n<image>",
        "The image is shown as you wish. \n<image>",
        "The image is generated. \n<image>"]
generation_data = []
count=0
for data_dict in tqdm(list_data_dict):
    if 'sam' in data_dict['image']:
        continue
    caption=data_dict['conversations'][1]['value']
    data_dict['conversations'][0]['value']= f"{random.choice(generation_q)} {caption}"
    data_dict['conversations'][1]['value']= f"{random.choice(generation_a)}"
    data_dict['task']='generation'
    data_dict['image'] = f"/public_data/ShareGPT4V/{data_dict['image']}"
    generation_data.append(data_dict)
    count+=1
    if count>=200000:
        print('reach 200k')
        break


with open('/public_data/jihai/data/share-captioner_coco_lcs_150k_generation.json', "w") as f:
    json.dump(generation_data, f, indent=2, ensure_ascii=False)


 62%|██████▏   | 770485/1246901 [00:00<00:00, 2115173.19it/s]

reach 200k


In [16]:
print(generation_data[:3])
print(len(generation_data))

[{'id': '000000125860', 'image': '/public_data/ShareGPT4V/coco/train2017/000000125860.jpg', 'conversations': [{'from': 'human', 'value': 'Can you draw a image of the following: In the image, a blue and white bus is parked at a bus stop. The bus is facing towards the right side of the image, ready to embark on its journey. The bus stop is situated on the right side of the image, providing a clear view of the surroundings. A yellow traffic light stands guard on the left side of the image, adding a pop of color to the scene. The sky above is a clear blue, dotted with white clouds, suggesting a bright and sunny day. In the background, trees can be seen, adding a touch of nature to the urban setting.'}, {'from': 'gpt', 'value': 'The image is shown as you wish. \n<image>'}], 'task': 'generation'}, {'id': '000000571342', 'image': '/public_data/ShareGPT4V/coco/train2017/000000571342.jpg', 'conversations': [{'from': 'human', 'value': "Please show a image of the following: In the image, a tranqu

In [1]:
import json
list_data_dict1 = json.load(open('/public_data/jihai/data/share-captioner_coco_lcs_150k_generation.json', "r"))
list_data_dict2=json.load(open('/public_data/ShareGPT4V/sharegpt4v/llava_v1_5_mix665k.json','r'))
list_data_dict=list_data_dict1[:200000]+list_data_dict2
print(len(list_data_dict))
with open('/public_data/ShareGPT4V/sharegpt4v/llava_v1_5_mix665k_with_generation.json', "w") as f:
    json.dump(list_data_dict, f, indent=2, ensure_ascii=False)

865298


In [2]:
import json
list_data_dict1 = json.load(open('/public_data/jihai/data/share-captioner_coco_lcs_150k_generation.json', "r"))
print(len(list_data_dict1))
list_data_dict2=json.load(open('/public_data/ShareGPT4V/sharegpt4v/blip_laion_cc_sbu_558k.json','r'))
list_data_dict=list_data_dict1[:150000]+list_data_dict2
print(len(list_data_dict))
with open('/public_data/ShareGPT4V/sharegpt4v/blip_laion_cc_sbu_558k_with_generation.json', "w") as f:
    json.dump(list_data_dict, f, indent=2, ensure_ascii=False)

200000
708128


In [3]:
2767*8*32

708352

In [9]:
import json

list_data_dict = json.load(open('./blip_laion_cc_sbu_558k_with_generation.json', "r"))
print(list_data_dict[:3])

[{'id': '000000125860', 'image': '/public_data/ShareGPT4V/coco/train2017/000000125860.jpg', 'conversations': [{'from': 'human', 'value': 'Can you draw a image of the following: In the image, a blue and white bus is parked at a bus stop. The bus is facing towards the right side of the image, ready to embark on its journey. The bus stop is situated on the right side of the image, providing a clear view of the surroundings. A yellow traffic light stands guard on the left side of the image, adding a pop of color to the scene. The sky above is a clear blue, dotted with white clouds, suggesting a bright and sunny day. In the background, trees can be seen, adding a touch of nature to the urban setting.'}, {'from': 'gpt', 'value': 'The image is shown as you wish. \n<image>'}], 'task': 'generation'}, {'id': '000000571342', 'image': '/public_data/ShareGPT4V/coco/train2017/000000571342.jpg', 'conversations': [{'from': 'human', 'value': "Please show a image of the following: In the image, a tranqu